In [1]:
from IPython import display
display.clear_output()

import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import roboflow

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics 8.3.111  Python-3.13.3 torch-2.6.0+cpu CPU (AMD Ryzen 5 8400F 6-Core Processor)
Setup complete  (12 CPUs, 15.6 GB RAM, 747.8/930.8 GB disk)


In [2]:
# Em apple sillicon, verificar se o MPS está disponível
import torch
print(torch.backends.mps.is_available())

False


# Treino de um modelo

In [14]:
# descarregar o dataset do roboflow, depois de etiquetadas as imagens e criado o dataset
# em Mac tive problemas de permissões e foi necessário dar permissões à pasta de conf. do roboflow:
# - sudo mkdir /Users/davidecarneiro/.config/roboflow (criar pasta onde vai guardar a conf.)
# - sudo chown -R davidecarneiro:staff ~/.config/roboflow (dar permissões ao meu user)
#roboflow.login()

# criar um ficheiro com a key da api do roboflow, ou simplesmente substituir abaixo
with open('api_key.txt', "r") as file:
    api_key = file.read().strip() 

rf = roboflow.Roboflow(api_key)

# substituir nome do workspace e do projeto
project = rf.workspace("albiononline-ia").project("albion-bot-e1ypw")
# se versão do dataset > 1, substituir pela versão correspondente
dataset = project.version(4).download("yolov8")
# WARN: necessário verificar os paths no ficheiro data.yaml, após este ser descarregado

loading Roboflow workspace...
loading Roboflow project...


In [20]:
# treinar o modelo
# lista de modelos pre-treinados disponível em https://docs.ultralytics.com/models/yolov8/#performance-metrics
model = YOLO("yolov8m.pt")  # carregar o modelo pre-treinado que se descarregou

# Treinar o modelo
#results = model.train(data='./data.yaml', epochs=20, imgsz=640, device=0) # intel/windows
results = model.train(data='./data.yaml', epochs=20, imgsz=640, device="cpu") # apple sillicon

Ultralytics 8.3.111  Python-3.13.3 torch-2.6.0+cpu CPU (AMD Ryzen 5 8400F 6-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=./data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, f

train: Scanning C:\Users\Francisco\Desktop\AlbionBotIA\train\labels... 273 images, 0 backgrounds, 0 corrupt: 100%|██████████| 273/273 [00:00<00:00, 2516.75it/s]

train: New cache created: C:\Users\Francisco\Desktop\AlbionBotIA\train\labels.cache
val: Fast image access  (ping: 0.10.1 ms, read: 127.320.6 MB/s, size: 63.8 KB)



val: Scanning C:\Users\Francisco\Desktop\AlbionBotIA\valid\labels... 27 images, 0 backgrounds, 0 corrupt: 100%|██████████| 27/27 [00:00<00:00, 3072.92it/s]


val: New cache created: C:\Users\Francisco\Desktop\AlbionBotIA\valid\labels.cache
Plotting labels to C:\Users\Francisco\Desktop\AlbionBotIA\runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to C:\Users\Francisco\Desktop\AlbionBotIA\runs\detect\train
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20         0G     0.8227     0.3698      1.013          1        640: 100%|██████████| 18/18 [04:24<00:00, 14.67s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:09<00:00,  9.13s/it]

                   all         27        227      0.971      0.943      0.979      0.642



20 epochs completed in 1.525 hours.
Optimizer stripped from C:\Users\Francisco\Desktop\AlbionBotIA\runs\detect\train\weights\last.pt, 52.0MB
Optimizer stripped from C:\Users\Francisco\Desktop\AlbionBotIA\runs\detect\train\weights\best.pt, 52.0MB

Validating C:\Users\Francisco\Desktop\AlbionBotIA\runs\detect\train\weights\best.pt...
Ultralytics 8.3.111  Python-3.13.3 torch-2.6.0+cpu CPU (AMD Ryzen 5 8400F 6-Core Processor)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:07<00:00,  7.98s/it]


                   all         27        227      0.973      0.926      0.981      0.645
          broken stone         16         71      0.966      0.915      0.977      0.595
                 stone         27        156      0.981      0.936      0.986      0.695
Speed: 1.8ms preprocess, 289.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to C:\Users\Francisco\Desktop\AlbionBotIA\runs\detect\train


# Inferência

In [ ]:
%%sql


In [3]:
# selecionar a melhor versão do modelo fine-tuned
model = YOLO("runs/detect/train/weights/best.pt")

NameError: name 'YOLO' is not defined

In [1]:
# prever em novas imagens
confidence_level = 0.1
input_path = 'captured_images'
output_path = 'detections'
class_names = model.names


for file in os.listdir(input_path):
    if file.lower().endswith((".png")):
        image = cv2.imread(os.path.join(input_path, file))
        results = model.predict(source=image, conf=confidence_level)  # gerar previsões acima de determinada confiança, e guardar imagens

        
        output_filename = f"prediction_{file}"
        output_filepath = os.path.join(output_path, output_filename)

        for result in results:
            result.save(filename=output_filepath)
            print("==== Resultados Previsão ====")
            print("Imagem: "+os.path.join(input_path, file))
            boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes (x_min, y_min, x_max, y_max)
            scores = result.boxes.conf.cpu().numpy()  # Score de confiança
            labels = result.boxes.cls.cpu().numpy()  # Índice da classe

            for i in range(len(boxes)):
                class_id = labels[i]
                class_label = class_names[class_id] if class_id in class_names else "Desconhecido"

                print(f"--- Objeto {i+1} ---")
                print(f"Class: {class_label} (ID: {class_id})")
                print(f"Coordenadas Bounding Box: {boxes[i]}")
                print(f"Confiança: {scores[i]:.4f}")
                print("-------------------")

            print("\n")

NameError: name 'model' is not defined

# Inferência em tempo real

In [35]:
import mss
import cv2
import numpy as np
import time
import pyautogui

In [37]:
# Carregar o modelo
model = YOLO("runs/detect/train2/weights/best.pt")

screen_width, screen_height = pyautogui.size()

# função que captura o ecrã e devolve a imagem
def capture_screen():
    with mss.mss() as sct:
        screenshot = sct.grab(sct.monitors[1])  # Capturar do monitor principal
        img = np.array(screenshot)  # Converter em imagem
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR) # Converter BGRA para RGB
        img_height, img_width, _ = img.shape

        # Opcionalmente, guardar a imagem capturada
        # timestamp = time.strftime("%Y%m%d-%H%M%S-%f")  # Include microseconds
        # img_path = os.path.join('captured_images', f"capture_{timestamp}.jpg")
        # cv2.imwrite(img_path, img)

        return img, timestamp, img_width, img_height

while True:
    img, timestamp, img_width, img_height = capture_screen()
 
    #results = model.predict(source=img, save=True, save_txt=True, conf=0.1) 
    results = model(img)

    # Extract detections (bounding boxes)
    detections = results[0].boxes.xyxy  # Bounding boxes (x1, y1, x2, y2)

    if len(detections) > 0:
        print(f"Detetou {len(detections)} objetos.")

        # Opcionalmente, guardar imagem com as deteções
        annotated_frame = results[0].plot()  # Draw bounding boxes on the image
        result_path = os.path.join('detections', f"result_{timestamp}.jpg")
        cv2.imwrite(result_path, annotated_frame)

        for i, (x1, y1, x2, y2) in enumerate(detections.tolist()):
            # Calculate the center of the object
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # converter coordenadas
            scaled_x = int((center_x / img_width) * screen_width)
            scaled_y = int((center_y / img_height) * screen_height)

            # mover o rato
            pyautogui.moveTo(scaled_x, scaled_y, duration=0.3)
            pyautogui.click()
            print(f"Moved to object {i+1} at ({scaled_x}, {scaled_y})")

            # Pausa breve, entre objetos
            time.sleep(0.25)  # Adjust delay as needed

    time.sleep(3)


0: 416x640 (no detections), 146.7ms
Speed: 3.2ms preprocess, 146.7ms inference, 0.3ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 4 tools, 147.5ms
Speed: 2.8ms preprocess, 147.5ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)
Detetou 4 objetos.
Moved to object 1 at (720, 594)
Moved to object 2 at (564, 517)
Moved to object 3 at (409, 440)
Moved to object 4 at (1168, 803)

0: 416x640 1 box, 4 coins, 2 tools, 122.9ms
Speed: 2.1ms preprocess, 122.9ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 640)
Detetou 7 objetos.
Moved to object 1 at (928, 411)
Moved to object 2 at (864, 443)
Moved to object 3 at (802, 474)
Moved to object 4 at (745, 910)
Moved to object 5 at (961, 721)
Moved to object 6 at (1114, 654)
Moved to object 7 at (943, 975)

0: 416x640 (no detections), 124.3ms
Speed: 2.3ms preprocess, 124.3ms inference, 0.3ms postprocess per image at shape (1, 3, 416, 640)


KeyboardInterrupt: 